### Testing all functions to get it working here

In [4]:
import numpy as np
import cv2 
import os
import copy
from sklearn.feature_extraction.image import PatchExtractor
from skimage import data, transform
import matplotlib.pyplot as plt

# 1. Implementation (Other implelemtation below)

In [2]:
def find_uglies():
    match = False
    for file_type in ['Negative']:
        for img in os.listdir(file_type):
            for ugly in os.listdir('Noimage'):
                try:
                    current_image_path = str(file_type)+'/'+str(img)
                    ugly = cv2.imread('Noimage/'+str(ugly))
                    question = cv2.imread(current_image_path)
                    if ugly.shape == question.shape and not(np.bitwise_xor(ugly,question).any()):
                        print('That is one ugly pic! Deleting!')
                        print(current_image_path)
                        os.remove(current_image_path)
                except Exception as e:
                    print(str(e))


def create_pos_n_neg():
    for file_type in ['Negative']:
        for img in os.listdir(file_type):
        
            if file_type == 'pos':
                line = file_type+'/'+img+' 1 0 0 50 50\n'
                with open('info.dat','a') as f:
                    f.write(line)
            elif file_type == 'Negative':
                line = file_type+'/'+img+'\n'
                with open('bg.txt','a') as f:
                    f.write(line)

# 2. Implementation (more promising, better understandable)

## Positive Samples

* prepare positive smples (download them!)
* make annotations to positive samples "opencv_annotations" for EXTRACTION of object from samples
* Reresize all logos to same size (smaller is better, as many logos as possible)
* Get 2nd annotations file for resized logos for creation of .vec file using "create_infodata_pos"
* create .vec file using "opencv_createsamples"

In [4]:
def ExtractObject(datafile, # annotation file
                  pathtowrite):
    #open datafile
    f = open(datafile)
    content = f.read()
    i = 1
    for l in content.split('\n'):
        words = l.split()
        if(len(words) >= 6): # coz sometimes the images have no region. 
            img_path = ' '.join(words[:-5]) # path the positive sample file
            img_path = img_path.replace('\\','/') # replace back-slash if you are window user
            img = cv2.imread(img_path,0) # read the read the sample using OpenCV
            logo = [int(w) for w in words[-4:]] #extract the logo
            x,y,w,h = logo
            if img is not None:
                logograb = img[y:y+h, x:x+w]
                # keep the size small to keep the training time short
                img = cv2.resize(logograb, (60,60), interpolation = cv2.INTER_AREA)
                cv2.imwrite(pathtowrite + str(i) + '.jpg', img)
                i = i + 1
            
def create_infodata_pos(imgfolder):
    for img in os.listdir(imgfolder):
        line = imgfolder + img + " 1 0 0 60 60\n"
        with open(bg_name_positives,'a') as f:
            f.write(line)

In [78]:
# use opencv_annotations()
# get annotations for extractions
# Extract object
path_training = "./train/"
filename_extraction = "annotations_for_extraction.txt"

bg_name_positives="bg_pos_test.txt"
bg_name_negatives="bg_neg_test.txt"

ExtractObject(filename_extraction,path_training)

#Annotation file of Positive Samples (after cut out)
# annotate again but with below function and not with "opencv_annotations"

create_infodata_pos(path_training)

## Negative Samples

- Download negative samples
- create X*Y sized patches from negative samples using "extract_patches"
- Create bg.txt file to list all negative files using "create_bg_neg"

In [5]:
def extract_patches(img, N, scale=1.0, patch_size=(60,60)):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches


def create_bg_neg(imgfolder,bg_name):
    for img in os.listdir(imgfolder):
        line = imgfolder + "/" + img + "\n"
        with open(bg_name,'a') as f:
            f.write(line)

In [88]:
images = []
source_neg = "Negative"
patches_root= 'patches'
subfolder = "streets"

for filename in os.listdir(source_neg + '/' )[200:400]:# iterate thru each image in a celeb folder
    filename = source_neg + '/'  + filename # build the path to the image file
    if(filename.endswith('.jpg')):
        img = cv2.imread(filename)
        if(img is not None):
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)

negative_patches = np.vstack([extract_patches(im, 75, scale)
                             for im in images for scale in [1.0]])



In [91]:
for image in range(0,len(negative_patches)):
    cv2.imwrite(patches_root + "/" + subfolder + "/"+str(image)+".jpg", negative_patches[image,:,:])


In [93]:
create_bg_neg(patches_root + "/" +subfolder,bg_name_negatives)

## Train Cascade

* using opencv_traincascade

In [6]:
def detect(faceCascade, gray_,  scaleFactor_ = 1.05, minNeighbors = 3):
    faces = faceCascade.detectMultiScale(gray_,
                    scaleFactor= scaleFactor_,
                    minNeighbors=5,
                    minSize= (30,30), #(60, 20),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )
    return faces

source_positive = "/home/jannik/Git-master/justice_detection/pos/"
cascade_path = "./cascade/cascade.xml"

def DetectAndShow(imgfolder,cascade_path):
    cascade_classifier = cv2.CascadeClassifier(cascade_path)
    for i in os.listdir(imgfolder):
        filepath = imgfolder + i
        img = cv2.imread(filepath)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        logos = detect(cascade_classifier, gray, 1.25, 6)
        for (x, y, w, h) in logos:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.imshow('positive samples',img)
        k = 0xFF & cv2.waitKey(0)
        if k == 27:         # q to exit
            break


In [ ]:
DetectAndShow(source_positive,cascade_path)